In [99]:
import csv
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np

In [216]:
df = pd.read_csv('out.csv', index_col=False)
label_df = pd.read_csv('train.csv',index_col=False)

In [220]:
df = df.sort_values('imageid')
label_df = label_df.sort_values('business_id')

In [221]:
def str_to_nplist(string):
    string = string.replace("[", "")
    string = string.replace("]", "")
    string = string.split(',')
    return np.asarray(string).astype(np.float32)

In [236]:
business_id = df['business_id'] 
mean_features = np.asarray([str_to_nplist(i) for i in df['features']])
labels = label_df[labels]

KeyError: 'business'

In [235]:
X = mean_features
Y = labels

NameError: name 'business_features' is not defined

,imageid,features
467,327,"[0.17010983938542967, 0.10384696330281246, 0.2..."
439,356,"[0.26085117495109905, 0.013256385631027252, 0...."
697,554,"[0.07980539975980111, 0.09745335333162383, 0.1..."
205,870,"[0.10381798062741131, 0.04730636430236313, 0.1..."
886,1072,"[0.18085780999177664, 0.2220855811334311, 0.23..."
963,1413,"[0.22175136238883889, 0.045844398056980186, 0...."
561,1884,"[0.09138064462645525, 0.12232422699054657, 0.1..."
726,1948,"[0.054851393389595264, 0.01131440473932211, 0...."
296,2225,"[0.08345166951374221, 0.03392444295925204, 0.0..."
819,3313,"[0.05089183630047491, 0.035007165029658116, 0...."
